In [ ]:
import pandas as pd
import numpy as np
import statsmodels as sm

long = pd.read_csv('patient_longitudinal.csv', sep='\t')
long['visit_date'] = pd.to_datetime(arg=long['visit_date'], format='%Y-%m-%d')
print(long.duplicated().sum()) #checking for dupes

0


In [ ]:
print(long.isna().sum()) #checking for NAs

patient_id       0
visit_date       0
bp_systolic      0
heart_rate       0
adverse_event    0
age              0
sex              0
bmi              0
smoking          0
diabetes         0
dtype: int64


In [6]:
ts = pd.DatetimeIndex(long['visit_date'])
print(ts)

DatetimeIndex(['2023-01-17', '2023-09-23', '2024-05-22', '2024-12-31',
               '2023-01-17', '2023-09-23', '2024-05-22', '2024-12-31',
               '2023-01-17', '2023-09-23',
               ...
               '2024-05-22', '2024-12-31', '2023-01-17', '2023-09-23',
               '2024-05-22', '2024-12-31', '2023-01-17', '2023-09-23',
               '2024-05-22', '2024-12-31'],
              dtype='datetime64[ns]', name='visit_date', length=4000, freq=None)
